# Data Design and Representation - Midterm
## Stella (Sute) Li

## Part 1

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import re
import time
import mysql.connector
import pandas as pd

In [27]:
# open the webpage and parse it into a json object
url1 = 'http://numbersapi.com/0..99?json'
url1_obj = requests.get(url1)
json_obj = json.loads(url1_obj.text)

In [33]:
# print each line of trivia for numbers 0~99
for i in range(100):
    if i<10:
        print('00{} -'.format(i), json_obj[str(i)]['text'])
    else:
        print('0{} -'.format(i), json_obj[str(i)]['text'])

000 - 0 is the atomic number of the theoretical element tetraneutron.
001 - 1 is the number of dimensions of a line.
002 - 2 is the number of polynucleotide strands in a DNA double helix.
003 - 3 is the number of witches in William Shakespeare's Macbeth.
004 - 4 is the number of chambers the mammalian heart consists of.
005 - 5 is the number of Korotkoff sounds when measuring blood pressure.
006 - 6 is the number of fundamental flight instruments lumped together on a cockpit display.
007 - 7 is the number of periods, or horizontal rows of elements, in the periodic table.
008 - 8 is the number of principles of Yong in Chinese calligraphy.
009 - 9 is the number of circles of Hell in Dante's Divine Comedy.
010 - 10 is the number of letters used in the traditional Snellen chart.
011 - 11 is the number of players in an American football team on the field at one time during play.
012 - 12 is the number of ounces in a troy pound (used for precious metals).
013 - 13 is the number of colonies t

## Part 2

## a) 
(modified) assignment 2

In [37]:
# get the web content
url = 'https://www.ebay.com/sch/i.html?_nkw=samsung+tv'
r = requests.get(url)
html = r.text

In [38]:
# save the search result to a file
file = open('ebay_samsung_tv_01.htm', 'w')
file.write(html)
file.close()

In [39]:
# save all the webpages from the search result
for i in range(1, 11):
    urls = 'https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_pgn=' + str(i)
    req = requests.get(urls)
    htmls = req.text
    if i <= 9:
        page_name = '0' + str(i)
    else:
        page_name = str(i)

    files = open('ebay_samsung_tv_' + page_name + '.htm', 'w', encoding='utf8')
    files.write(htmls)
    files.close()
    print('page no. '+ str(i) +' is saved')
    time.sleep(10)

page no. 1 is saved
page no. 2 is saved
page no. 3 is saved
page no. 4 is saved
page no. 5 is saved
page no. 6 is saved
page no. 7 is saved
page no. 8 is saved
page no. 9 is saved
page no. 10 is saved


In [40]:
# locate sponsored items and print out all the links
file_name = "ebay_samsung_tv_"
for i in range(1,11):
    if i < 10:
        page_name = '0' + str(i)
    else:
        page_name = str(i)
    with open("ebay_samsung_tv_%s.htm" %(page_name), 'r',encoding="utf8") as file: 
        html = file.read()
    soup = BeautifulSoup(html, "html.parser")
    page_ind = i
    print('Page %d sponsored items:' %(page_ind) +'\n' )
    for i in soup.find_all('div', attrs = {'style':'display:flex;'}):
        pattern = re.compile(r'.*[S].*[P].*[O].*[N].*[S].*[O].*[R].*[E].*[D].*')
        text = i.getText()
        if pattern.search(text):
            print('    Item: ' + i.parent.find('h3').getText())
            print(i.parent.get('href'))

Page 1 sponsored items:

    Item: Samsung 7100 UN40NU7100F 40-inch 4K Ultra HD LED TV UN40NU7100FXZA 40" 7 series
https://www.ebay.com/itm/Samsung-7100-UN40NU7100F-40-inch-4K-Ultra-HD-LED-TV-UN40NU7100FXZA-40-7-series/392658887563?epid=21018486682&_trkparms=ispr%3D1&hash=item5b6c4b238b:g:t8YAAOSwIW9eLzSe&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qVLY1AmmlDIgbTnIsGkvdVmgFSMtmtXV06GE7Iw8OaBVUA3pqAr1l0PYxixkWCGCDdo2CbZiHNhGUMnECgjN8htaa5RxeOic8h1rSIo6McqsF7lRXYW7b5loiAWjQ2ITmuXZK6%2BcQRsIuJZUjT4oMnPfcOjjlk4YstJRWc%2FrrzZe4HU39tSqEBf20uFrAOxzX6WtbODHFb%2BJdQMwzKMuzNaiRq5%2Bacl5GD%2Bs8FFiy3WPMO6YdOVCfyrxJdIOQkr7qVkNgG6ratfy1LO3ylLYV4d1EiUXqOSslKN42A%2B8xX5IAR%2FFep7pYC%2BjYYsfgXp3NDDegcDIM2tAfywR%2BYZdQji2SCJuK6Y%2BUVRI6lFtfWLI7Sq4KP2BJASRNIFRpMrRAcHkOAjZijn6c79OOg81UC7kmh0VeTsjR6Kn4g4Ykl8JvhZyDriQ7CxnoLJ7u4v6LkUBRW3UMSd%2Fa5teuL7fKQy%2BmKlzUBPW3Yej0hxZtQE3fhu7cB%2FthgaMWTibHHmaeLqc55qDSPwdUatK18HxxbSzmLf2ZitSwu9dwdzCyzkRgkVrPcQjUKF6%2BuhfjExcSjMCfDfwMUGtciR%2BzVegSsqz%2BOZEYiryU93z%2BMd%2FmAb4Am

Page 2 sponsored items:

    Item: Samsung Series  2013 LED TV (FH5000 Series) 39’’
https://www.ebay.com/itm/Samsung-Series-2013-LED-TV-FH5000-Series-39/233478374293?epid=239307368&_trkparms=ispr%3D1&hash=item365c654395:g:Ju0AAOSw5G9eL9j4&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWI8HPg%2B4alJz25LRV27M4Be%2BkVi4lWXEJmy3bsT2UOATrBp%2FBxU5%2F67p%2FzCANdCfyjT2s0MKy3tzC%2BvbY1tkXdC2YjHolOoIQQuI%2FCVZ4rKXYbS8C7xIKEtS6%2Fh4orp16lw7q3lBTjg%2BWB3JO2voyA2g1EGtypWIuOvUxHtekaleGy5hStXrIMhjMbTuUGLNTYBNaywcTM6O6UuDrj9C4tbOkA4caRwFMkX%2FNFxQIjtOzpOG%2BFuk45%2FzKK2TRQ9Ob9xqPGm3wNzVKPm2EoNSAA9QnKJdO55M7JdQEMmmTFFXC27jzql2tSpd0fUe8pEauHxO41puCz1nCQDZuZvo307sBUuDa6dJNuex4rrW%2Fmp9M8PLun6k5TxgLiMSsBuSFWUR%2BijwT8cAMjtPJU7GLlOOdfRsTQ3w60hGxino5qHa7%2FgbpkSvdj141SbYThIUvtD1WTzcmkQ7ldVlUVmrLvd1gb6XYz2N02h%2FYCzUjH%2FRmAMCMiQNpQu9gSNqUeB%2BEg6aswZIjDLsyjZFTJz1NrhOkWQMvzpBdHhNjneVvvfblmcFmmqZFVyldNQ7EolChmCEkJTkYsnWPpC6UIjEkwPE4PyVOG66evt31sigBSBZG4VpCJnj6a1XjcYlVfPV7D8%2FOiVR4weTjRupVelLhF13e2hEBwOiL%2BkocW%2FLMIo

Page 3 sponsored items:

    Item: New Replacement TV Remote Control For Samsung Television UN65NU6950FXZA
https://www.ebay.com/itm/New-Replacement-TV-Remote-Control-For-Samsung-Television-UN65NU6950FXZA/283564369190?_trkparms=ispr%3D1&hash=item4205c0e526:g:uokAAOSw9yldP41W&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qW6kkLr0WgPzUBMVRa1FBSUUPuOMLZAYd9eO4%2FV90qjBycTtwWKhThsCXvID%2BLpkkTADxo3lLPog8B%2FhpkGqzJnSSt81bEn1qUfwXwvJiwmRvSLm71fncslanF5ULZbOZlZNOQyr4eH75sfy24xhFe%2BY3AWG6DvrpeLFGU4S7xxbuiIAPu6%2FVH2a4lcJLm%2BdbTp7Di9bDSngdsPzXD7skUQfQlNCW6QvW6Ec396x6U%2B%2BjCOAKR4hWf4QBRT1rwR%2FPl%2BFgmKRWJqX47bTCG5yQcT3bd9tJFEllIcudz4Gp783AZI6KHsLrBGxv1vqB777AcXsm4GgJVgMJl2a9OgpOy3Y4fkIOeMcg995vdgPoPV%2Fj6wam1lUpxs34ajzTV3iGAcfddJKvI6NFP0qLHiQNet%2F36eGblUY1c%2BhMyTGAcbwr0VWh2Mjwr9QYLAm4yVAu3%2BSF4nAoYYRV8X61oBKShmyjpdnZyfsjbtJwaUXTLpcTBB2fRQS0gxtogdS9ujjLW68QMMgoNcjoVWW5HtClNk%2FSM58JSY9PZGDAp7kfhmwlSpwqtG5CFOOnlIOAMdlnoMU9%2FW3lwQkFZjM%2B52AIsGUd1mfu3enyjgzBuCdBxnG2WDzj7lKR8hICnywYHjK21J90CsVfE8hjbHq

Page 4 sponsored items:

    Item: SAMSUNG UN50RU7100 LED TV
https://www.ebay.com/itm/SAMSUNG-UN50RU7100-LED-TV/283674125651?epid=17028552154&_trkparms=ispr%3D1&hash=item420c4ba553:g:ybEAAOSwZi5dy9Hi&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qVvgNb2M4Ajb49S6kwrYHkdjrJ2vgiDB%2Bo0MGrSTU1mw7YHmRMZS2fQx30TZxx8tnbmPfThOey7J7Aj2W3DI4VSYcxoWdWFek5hDOVP2h%2BrSqjnoYqmmwAEnNvWsmksKWUNfEmIQ%2B1bU0QLerc0hHK83N%2FmUpERf4PvCjmndGy91%2BmokVEtBO3Sy38tCj4PMZj2sjoDZJkPptJOklWwsUrtiDw7TdeGbEQygX8U6ha4vodPcnZXOMHNMYg40uKh8n%2BUZTGX63yUvQQga6oHf4ftj%2B%2FH8Yjg76Ad24S%2BLFHyb%2Be65jctZ6GdP39drIZdPxYCQiC4zHNXa2%2FK93CS0J4Ng4kHaZoaPr49YxpkP1AhwJPnVFx5oVh217G%2FW%2FBRWqtXEHfI6g27dj9OA5495Hy91NgeiKmKVwdyqiwzgxg4wlniqamyuf%2FiU5AHEEmRE%2F0zn%2Fy9IY%2Boft1I8NcbxG0BvdZnSRU39XfNTBdgHouKxdC%2BJ0ICtEmmvc%2FbsWZe0a5J7%2BnSHPNa1yPZPQx2F9tDEaNmesn9POf956NVYdvqUcdgOub9eMp%2F5RBcMa0BAgAiZyJRW23KPPv1v0WRRxL4oLPlbCP0fC4DNGXYjwm8%2F6YbqgVTmy96foTY4j72Ud%2Fi6An36ZDir6j%2B8buq0YO6MqXvWUgKDSO8tckOiN%2Be3%2BM%2FQAy7anSMpw7ArjMsjh7ThVyD

Page 5 sponsored items:

    Item: Samsung 6300 Series UN40F6300 40" 1080p HD LED LCD Internet TV bundled w/ Remote
https://www.ebay.com/itm/Samsung-6300-Series-UN40F6300-40-1080p-HD-LED-LCD-Internet-TV-bundled-w-Remote/152747447936?epid=152229793&_trkparms=ispr%3D1&hash=item2390751680:g:bo4AAOSwB3BZ5kPB&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWMobHQ%2F3lhTsyRpdOFoMCXs5KYx%2BHHrjj3zdZsv6KA1unLsLrkJOe1Ys1OHPJzhnIJ%2B28VfksrHFPrR78Rki2jdj1Uwe9570bwqPZPXiOLOuKGn%2FNMfRE7w8luX3MdsXlxgqGdClvxAG8fNH6D6YZe7bkt1%2FC627sQ25fVWOJmrEMgMw37nMSs%2FS2gMf6LAf3xwGvC4OsHR8Tcc%2FZhUk7Kiq49i%2B7Ory1G0kUtRHSjWVWYKBxFFgKI%2F53M%2FlIHwKDI9gqbhGvC6xedm1RpC4yIENVfUlPKRM2r1HNt1Td1zsrdEqKrBHrSkcg5ttGf%2FiUOW5SOYHE64EOXoeZq86tdEQJo9mOaG0QJLLAPvRkjf0xuP6qdofbK5%2F4%2B15tTKcA5YFiJCAfVejnLtjvmzZAdvTwqcjL%2FjVUmCCTQhrywXYkwy0m7cp8z5EP6stasMm1SbCVuCJw%2FFqRNiWcGvsXlTWzrnwbrRDxkf%2BHurNdhX%2FQtsuascnOeN9dUJXmDoatWeeI24nQd0j%2FbrzS1BLE6ODP0j7yyjvHAsisi1vX%2FRG3yhnKTalFAor3tzwY2rENRbkYEF5JMhDoToYJ4kn6u6iblesUCLEIAwMH1ofi3at

Page 6 sponsored items:

    Item: Samsung Series  2013 LED TV (FH5000 Series) 39’’
https://www.ebay.com/itm/Samsung-Series-2013-LED-TV-FH5000-Series-39/233478374293?epid=239307368&_trkparms=ispr%3D1&hash=item365c654395:g:Ju0AAOSw5G9eL9j4&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWI8HPg%2B4alJz25LRV27M4Be%2BkVi4lWXEJmy3bsT2UOATrBp%2FBxU5%2F67p%2FzCANdCfyjT2s0MKy3tzC%2BvbY1tkXdC2YjHolOoIQQuI%2FCVZ4rKXYbS8C7xIKEtS6%2Fh4orp16lw7q3lBTjg%2BWB3JO2voyA2g1EGtypWIuOvUxHtekaleGy5hStXrIMhjMbTuUGLNTYBNaywcTM6O6UuDrj9C4tbOkA4caRwFMkX%2FNFxQIjtOzpOG%2BFuk45%2FzKK2TRQ9Ob9xqPGm3wNzVKPm2EoNSAAeMbpCU7hpMxQCpZJNYvO8h9rzP7QicvYLzLjBmCWVjwWVnRj0A5NBh%2FVhjS2tpP0ngeBdRM4mUp%2BnXIDfQXR9smG7QtrLOrHgSKLHjkD8Lzz6HbhE0PGg%2FQlWBtErlkoTIK%2BENeabaEg4XYFKF5VXyr%2F3jjczLJ4FxWQnWufKYlVrUX1BldXsa8Za7gOoxWH7Tfy4dToolQi%2BGx%2FDkjrOXvYv7eORqKxLrIqsfX%2FTGqAdgnQQ3r15aH3ga0XJ9lHgGZgA5mx3PUutXhGskI2brjY3vOxca4lyXzFhRTW9GPxitHuhLm%2Fwdyffbi29g3ZISzgnvg1Wj%2FM%2BNZdsTvD8J%2FAgOvDu3WDPpnD7RI25XchgFTSsDuceopXUcoHd%2BdY0LpDPXll1Mc%2

Page 7 sponsored items:

    Item: Samsung UN65NU6900FXZA 65 inch 2160p LED Smart TV
https://www.ebay.com/itm/Samsung-UN65NU6900FXZA-65-inch-2160p-LED-Smart-TV/254500492540?epid=9025091411&_trkparms=ispr%3D1&hash=item3b416954fc:g:pQ8AAOSwRbddVBoA&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWVdtJIsuv6sDmOiEp0bS9iylee3pa0rwCjxFlO7WXIWRHUjJAwY3RPxsjgb%2BRs6YGPhA0SUfjGCxH9S6EIZmZhC1cv%2FYMAO7pE55LEd2tDeWrfSIymEOk4PsFWskhCzNBtfyB%2BKUG7bavZ2%2FPjFixGvuNjrNzmfQXkgsIqYcSbacUzcU%2BwrUJKf1y%2B1bwxI4duK2BOtXxYOFu9B8fb3LanE4n%2B8EqGNFsiqaj3ka0QIOqwmhbLX0uIDsb896oLFUqkmXIsu%2BS3UGayOtpoZKPqz66HnGIoRmFMKFQEhipfgosR2NDKXpMWUmMHQQFBoY02gNw%2BKNLWLzn2J%2BxdDqmttiGwestZDcgp77DCu0sxF%2Bmhn%2FBltOMHEhJg5JG9YKrBBDPq3DsPFnP7mDbl%2BDVc59PRNrjgiRMq5O8geZ1mC8BHqKqtSrh86TcAaou91n4YJ%2Fp9MKk0th425pxeTdy33CkxO9ANSDw3wPUAfPEaDBOBOQfyp9CQ%2F84H7Q%2B4%2F2Lij3ESdX9o6D56aasFLGRsg1JyWOn3LCJAdeX18tF%2B4wnGzWzpgaAdG4XjzYZUFnoq46OUIJtJXGJaoK7Q2NAeK0KvMTGwWuy2wxn1rX8FuqvaWULOh6joJbqnXnRWa%2BGkTYWDaBcKI7KdhYFeKjwyiVN%2FpKugQYozsgu

Page 8 sponsored items:

    Item: Samsung TXK-3276 32" Color CRT Television SEE NOTES 
https://www.ebay.com/itm/Samsung-TXK-3276-32-Color-CRT-Television-SEE-NOTES/333149321360?_trkparms=ispr%3D1&hash=item4d913f3890:g:LPsAAOSwJRleOKtY&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qUHuoC5QLxhsZu2pifR33Jch0zPudQTQzCQLjLXONDHbpul9tOMMmlP8c8CyLcTqyQGb8vj%2BuSn%2Bzqo%2FNyBkHGrXINnXdS6RMjjr1JlS6c3coeLHFmohrs3ihZqM0BO5rvOXlknveDK4T4DOacZdwLqtcrKfkfI8cWMQJJ3oOo2oH2w%2Fx%2BYgrwL36szoMl9VZkFfP2BdAWuk4sZh9NpPMvblu%2F9fbnBd%2Fc0FeZmnhdE2wOiV2vWZcQ%2FyHPggyU98MQejsxTJJ7XQJWrT5lXCUwTqJdZsMpjUXwXxRc69OOao8BvLUQirzezFLeRxiQneAh0YlqzX7VNxtq1VBp%2F%2FmJbtASqD95OffmlA059QqUyVV5xhpcFE5F0V8pJLFaYKHiyB9rZ4fcdt1qzcA6Ez5CMJXQeCIS%2BwgJoxWHwNwQ5oG54PKyLK4hyD%2BgYNr%2B%2BTSjQe%2FNMjdeTTTJPoTMutfNW%2FfH9XRFEtTf3WarUFcnns0DTmOpp8FNHxAuqv8ODxm%2BnWdYAaQvoqR2V61b%2FlCnoUsgl8tc3bYicoObCtGdjOYLJHc51XiAtg0wtpCPx2k24crO3QpILj9vu%2BhGkmuclG7uTLHrJrqI1A3Jt4AFhfHgSBy6bxAMxFkv0d1BfoSWV2xL7c8hw0qlKT4Eoc9st5mZPMQ80aeNuW035WIruYIZME%2FH

Page 9 sponsored items:

    Item: Samsung 55" Class Q7FN QLED Smart 4K Ultra HD TV (2018) QN55Q7FNAFXZA
https://www.ebay.com/itm/Samsung-55-Class-Q7FN-QLED-Smart-4K-Ultra-HD-TV-2018-QN55Q7FNAFXZA/133313772823?epid=6017345470&_trkparms=ispr%3D1&hash=item1f0a1ebd17:g:-nsAAOSwu4peKhDD&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qXiPC%2BckW9rHPAml9lQ8eYGuWg53pmx0Mhnk1Th4czdvGQCxqvyY5S3lWYKSutz8900DvkODKoqeEspTYCJfxFREkiN3zxciuGiDsTcfqEgqRrZFfaB92IJAzxlCLjOzIDL4xoUneSlD3yeyiYVq%2BiQ4ZA9kU0rcDJ8805j96yH1c9%2FpKHwHBNmCkU9%2F%2FgibkHH%2BVBBRjQI3GgunTl0%2B5qgrnzqTLxFB0Bf4i4EqIy%2BiAuChJiy2EILW98OD7MbgLQxq1OExugh40QGbMgN%2FYcArSMRIZY0A%2B%2FNj6Jra7YY5ZuK95XNlgHCuZY4Qzuaz5K%2Bbi36pFiWgmuh29irXaXJkWJ%2F8MYoY2IJNTCIkebVVkU4vv%2BCc09ytv22%2BpatDI6UR3N%2Fmz6WVbLU8Ze9fonUIRT1oLrXjUopjPAFdzIiRBVaFtM89rUCNH8HKNDyHfxDxM6FRI1JXgnrOTbP9iAHwfsWRlkYRdCRkXUFEHA835qKcZDdJMHZOmEFZPAY65QO%2ByelfbDdG%2FEr7DnLmHF5XlPdTIQ6kggJ3%2FNzXg7WdLtXTWsgIOrBhHN9F3ilqYW3ZhdZUQyO5ydv5IcXuNKKAfG%2FTRKG78%2BcuyH5osJzmTX7DgbQX40hL1KilrFQ

Page 10 sponsored items:

    Item: Samsung 40” UN40J6200AF Flat Screen TV No Base
https://www.ebay.com/itm/Samsung-40-UN40J6200AF-Flat-Screen-TV-No-Base/402044833318?_trkparms=ispr%3D1&hash=item5d9bbd4e26:g:9jsAAOSw4qZeJjx0&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qUmacwdwU3dd0R0tbEzFIAhw2s9ekOutdZAs4FPSzSmXxMZoCVOuSVY9T1UeZBJ4ptTywz2mGQX9aakaUDsqSRPRQ15d44arc6Yzge689wHM6DypsGVAWciDYc10YKTilQCevNBnUp1QvatQkPLRw%2BRMuwD0Lypt91mKTCGbFxK8lFcmfzoXde66FzPXASElBr812KShGwNjsx4ryaih7AWW5Yd8Zfud3uTEQejFWCa2mAtuGPvcUnPeKe4NH%2FLLyxQ%2BTuLRJ%2BxtTAYZElgJR5C9Mp9IX61IDZIMONrJN1M0gaJwr5FH9WQ9sWIFs1tarXJfPOn5gQ6c98GQtRhl1xVlEeCts7ufs2chB2zf8g9%2BFq3Or0eBwfHQhuyWcibLd5Ae5kdsOMevyYKbYciMtUpOclvNcti6BVMB7SYlZUjDUBH8NrOTQYbuAZ9tD%2FmHplQABDjk0FLxaXGKtShlBEdUxN2pi%2BPKH5EJZj%2FFGSGdgduYuiihHKMI%2BUNq9CiiJfDlWPaqPiO%2FHMTbM6FHRbNw50%2FniHMAKMvcjwGVG%2BmRR9914bmM8bBEy3MOA31ItJI98Ay4X1bLG6i7DhG3qxRLJRZdrDSSJ5rFOiD4OrObCzCtH7z%2FXNltgUhOqP%2B4u3DRefPG1xfqdCV7JBngjnq0CN9Y5jzvUPx6uExEKA1y8k9%2BiaHmE8ENiYrR%2Ft2OItpW

<b> How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page? What is the GET request's variable name corresponding to items per page searches?


In [36]:
## For buy-it-now items:
# I would modify the url to be the following: 
# https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1
# i.e. adding '&LH_BIN=1' to the end of the url.
# The GET request's variable name corresponding to buy-it-now searches is 'LH_BIN'.

## For 100 items per page:
# I would add '&_ipg=100' to the end of the url. 
# The url would be 'https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_ipg=100'.
# The GET request's variable name corresponding to the number of items per page is '_ipg'.

## b)
<b> Modify your previous code to search for buy-it-now listings of <keyword> and limit the number of items to 100 per page. Use this guide to select the right <keyword> assigned to you:
(my last name begins with a L)
- If your last name on Canvas starts with G-L use <keyword>: "playstation 4 slim"

In [211]:
# specifying search key word = playstation 4 slim, buy-it-now items only, items per page = 100
url2b = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&_ipg=100'

In [212]:
# create two files to store all the links
file_sponsored = open('sponsored.txt', 'w')
file_nonsponsored = open('non-sponsored.txt', 'w')

for i in range(1,11):
    # parse the webpages and find sponsored items
    url = url2b + '&_pgn=' + str(i)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")

    pattern = re.compile(r'.*[S].*[P].*[O].*[N].*[S].*[O].*[R].*[E].*[D].*')
    
    # find urls of sponsored and not sponsored items and write into the files
    for elem in soup.find_all('div', attrs = {'style':'display:flex;'}):
        text = elem.getText()
        if pattern.search(text):
            sponsored_url = elem.parent.get('href')
            file_sponsored.write(sponsored_url + '\n')
            
        if pattern.search(text) == None:
            unsponsored_url = elem.parent.get('href')
            file_nonsponsored.write(unsponsored_url + '\n')
        
            
file_sponsored.close()
file_nonsponsored.close()

## c)
<b> Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item- id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [213]:
# create the two folders 'sponsored' and 'non-sponsored'
cwd = os.getcwd()
path_sponsored = os.path.join(cwd, r'sponsored')
path_nonsponsored = os.path.join(cwd, r'non-sponsored')
try:
    os.mkdir(path_sponsored)
    os.mkdir(path_nonsponsored)
except:
    print('directory already exists')

In [214]:
# store html code of webpages
def webpage_to_file(url):
    request = requests.get(url)
    html = request.text
    return html

In [215]:
# how to find the id of items from the url
id_pattern = re.compile('/\d{12}\?')

# save all the webpages from the links
def save_ebay_pages(sponsored = False):
    counter = 0
    if sponsored == True:
        file_name = 'sponsored.txt'

    else:
        file_name = 'non-sponsored.txt'        
        
    with open(file_name, 'r') as file:
        if sponsored == True:
            os.chdir(path_sponsored)
        else:
            os.chdir(path_nonsponsored)
            
        for line in file.readlines():
            counter +=1
            url = line
            item_id = re.search(id_pattern, line).group()[1:-1]
            
            try:
                html = webpage_to_file(url)
                
            except:
                print('error at link no.', counter, ': \n', url)
                time.sleep(2)
                html = webpage_to_file(url)
                continue
                
            ebay_html = open(item_id +'.htm', 'w', encoding='utf8')
            ebay_html.write(html)
            ebay_html.close()
                
            time.sleep(2)       

In [216]:
save_ebay_pages(sponsored=True)

In [217]:
os.chdir("..")

In [218]:
save_ebay_pages(sponsored=False)

## d)
<b> Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).
In your code, highlight the selector command you choose to obtain each element using comments.

In [237]:
# navigate to the path of non-sponsored items
def navigate_to_path_nonsponsored():
    if os.getcwd() != os.chdir(path_nonsponsored):
        os.chdir(path_nonsponsored)

In [238]:
# navigate to the path of sponsored items
def navigate_to_path_sponsored():
    if os.getcwd() != os.chdir(path_sponsored):
        os.chdir(path_sponsored)

In [671]:
navigate_to_path_sponsored()

In [673]:
navigate_to_path_nonsponsored()

In [684]:
# store all items in a list of dictionaries
# initiate the list
ebay_items = []
index = 0

In [491]:
# to find the price of items
price_pattern = re.compile('\$[\d\.,]+')

In [685]:
def store_ebay_items(sponsored=False):
    global index
    
    if sponsored == True:
        navigate_to_path_sponsored()
    else:
        navigate_to_path_nonsponsored()
        
    for filename in os.listdir():            
        f = open(filename, 'r', encoding = 'ISO-8859-1')
        html = f.read()
        soup = BeautifulSoup(html, "html.parser")
        
        # the id of the item read from the filename
        item_id = filename[:-4]
        # append a dictionary for each item to the list and update the id for the item
        ebay_items.append({'item_id': item_id})
        # update whether the item is sponsored
        if sponsored == True:
            ebay_items[index].update({'sponsored': True})
        else:
            ebay_items[index].update({'sponsored': False})

        try:
            #### seller name ####
            seller_name = soup.find('span', class_='mbg-nw').text
            ebay_items[index].update({'seller_name': seller_name})

            #### seller score ####
            seller_score = soup.find('span', class_ = 'mbg-l').find('a').text
            ebay_items[index].update({'seller_score': seller_score})

            #### number of items sold ####
            num_items_sold_object = soup.find('a', class_ = 'vi-txt-underline')
            if num_items_sold_object == None:
                num_items_sold = 0
            else:
                num_items_sold = num_items_sold_object.text[:num_items_sold_object.text.index(' ')].replace(',', '')
            ebay_items[index].update({'num_items_sold': num_items_sold})

            #### whether best offer option is available ####
            best_offer_object = soup.find('div', class_ = 'vi-bbox-dspn u-flL lable boLable')
            if best_offer_object == None:
                best_offer_available = False
            else:
                best_offer_available = True
            ebay_items[index].update({'best_offer_available': best_offer_available})

            #### title ####
            title = soup.find('h1', class_ = 'it-ttl').text[len('Details about  \xa0'):]
            ebay_items[index].update({'title': title})

            #### whether returns are allowed ####
            returns_text = soup.find('span', id = 'vi-ret-accrd-txt').text
            if 'Seller does not accept returns' in returns_text:
                returns_allowed = False
            else:
                returns_allowed = True
            ebay_items[index].update({'returns_allowed': returns_allowed})

            #### condition ####
            condition = soup.find('div', id = 'vi-itm-cond').text
            ebay_items[index].update({'condition': condition})

            #### item price ####
            item_price = soup.find('span', id = 'prcIsum').text
            if item_price.startswith('US') == False:
                item_price = soup.find('span', id = 'convbidPrice').text
            if item_price.startswith('US') == False:
                item_price = soup.find('span', id = 'convbinPrice').text

            #### shipping cost ####
            shipping_cost = soup.find('span', id = 'fshippingCost').contents[1].text
            if shipping_cost.startswith('$') == False and shipping_cost != 'FREE':
                shipping_cost = soup.find('span', id = 'convetedPriceId').text

        # item price and shipping cost on different pages are in different formats
        except:
            try:
                item_price = soup.find('span', class_ = 'cc-text-spans--BOLD').text
                if item_price.startswith('US') == False:
                    item_price = soup.find('span', id = 'convbidPrice').text
                if item_price.startswith('US') == False:
                    item_price = soup.find('span', id = 'convbinPrice').text
                shipping_cost = soup.find('a', id = 'e3').text

            except:
                try:
                    item_price = soup.find('span', {'itemprop': 'price'}).text
                    if item_price.startswith('US') == False:
                        item_price = soup.find('span', id = 'convbidPrice').text
                    if item_price.startswith('US') == False:
                        item_price = soup.find('span', id = 'convbinPrice').text
                    shipping_cost = soup.find('strong', class_ = 'sh_gr_bld').text
                except:
                    try:
                        item_price = soup.find('span', id = 'convbinPrice').text
                        if soup.find('span', id = 'fShippingSvc').contents[1].text != None:
                            shipping_cost = soup.find('span', id = 'fShippingSvc').contents[1].text
                        else:
                            shipping_cost = soup.find('span', id = 'fShippingSvc').text
                    except:
                        shipping_cost = None    
        # after locating the item price, convert it into a clean format
        price = re.search(price_pattern, item_price).group()[1:].replace(',','')

        # clean the shipping costs
        if shipping_cost != None:
            if shipping_cost.startswith('$'):
                shipping = shipping_cost.replace('$','')
                shipping_computed = True
            elif 'free' in shipping_cost.lower():
                shipping = 0
                shipping_computed = True
            else:
                shipping = None
                shipping_computed = False
        else:
            shipping = None
            shipping_computed = False

        ebay_items[index].update({'item_price': price})
        ebay_items[index].update({'shipping_cost': shipping})
        ebay_items[index].update({'shipping_computed': shipping_computed})

        f.close()
        index +=1

In [687]:
store_ebay_items(sponsored = True)

In [688]:
store_ebay_items(sponsored = False)

## e)
<b> Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d) into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non- sponsored. If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [631]:
# connect to my mysql database
mydb = mysql.connector.connect(host='localhost', user='root', connect_timeout=5)

In [632]:
cursor = mydb.cursor()

In [633]:
# create the database and table
cursor.execute('CREATE DATABASE eBay')

In [692]:
cursor.execute('USE eBay')
cursor.execute('DROP TABLE IF EXISTS eBay_items')

In [693]:
create_table_script = ('CREATE TABLE eBay_items ( '
'item_id BIGINT, '
'sponsored BIT, '
'seller_name VARCHAR(255), '
'seller_score INTEGER, '
'num_items_sold INTEGER, '
'best_offer_available BIT, '
'title VARCHAR(255), '
'returns_allowed BIT, '
'item_condition VARCHAR(255), '
'item_price INTEGER, '
'shipping_cost INTEGER, '
'shipping_computed BIT);')

In [694]:
cursor.execute(create_table_script)

In [659]:
insert_script = ('INSERT INTO eBay_items (item_id, sponsored, seller_name, seller_score,'
                                            'num_items_sold, best_offer_available, title,' 
                                            'returns_allowed, item_condition, item_price, shipping_cost,'
                                            'shipping_computed) '
            'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')

In [698]:
# insert each item into the table
for elem in ebay_items:
    # find and create all the corresponding variables
    item_id = elem['item_id']
    sponsored = 1 if elem['sponsored']==True else 0
    seller_name = elem['seller_name']
    seller_score = elem['seller_score']
    num_items_sold = elem['num_items_sold']
    best_offer_available = 1 if elem['best_offer_available']==True else 0
    title = elem['title']
    returns_allowed = 1 if elem['returns_allowed']==True else 0
    item_condition = elem['condition']
    item_price = int(float(elem['item_price'])*100)
    shipping_cost = None if (elem['shipping_cost']==None) else int(float(elem['shipping_cost'])*100)
    shipping_computed = 1 if elem['shipping_computed']==True else 0
    
    values = (item_id, sponsored, seller_name, seller_score, num_items_sold, best_offer_available, title,
          returns_allowed, item_condition, item_price, shipping_cost, shipping_computed)
    # insert each item into the table
    cursor.execute(insert_script, values)
    mydb.commit()

## f)
<b> Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and sd for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations.

In [716]:
# compile the query to find summary statistics
summary_query = ('SELECT sponsored, item_condition, '
'avg(seller_score), max(seller_score), min(seller_score), stddev(seller_score),'
'avg(num_items_sold), max(num_items_sold), min(num_items_sold), stddev(num_items_sold),'
'avg(item_price), max(item_price), min(item_price), stddev(item_price), '
'avg(shipping_cost), max(shipping_cost), min(shipping_cost), stddev(shipping_cost),'
'avg(best_offer_available), max(best_offer_available), min(best_offer_available), stddev(best_offer_available),'
'avg(returns_allowed), max(returns_allowed), min(returns_allowed), stddev(returns_allowed),'
'count(item_condition) '
'FROM ebay_items '
'GROUP BY sponsored, item_condition;')

In [718]:
cursor.execute(summary_query)

In [719]:
result = cursor.fetchall()

In [726]:
# store the summary statistics into a table and convert into a pandas dataframe
summary_table = []

In [727]:
for row in result:
    summary_table.append(row)

In [730]:
# update the column names
summary_df = pd.DataFrame(summary_table)
summary_df.columns = ['sponsored', 'item_condition', 'avg(seller_score)', 'max(seller_score)', 'min(seller_score)', 'stddev(seller_score)',
'avg(num_items_sold)', 'max(num_items_sold)', 'min(num_items_sold)', 'stddev(num_items_sold)',
'avg(item_price)', 'max(item_price)', 'min(item_price)', 'stddev(item_price)',
'avg(shipping_cost)', 'max(shipping_cost)', 'min(shipping_cost)', 'stddev(shipping_cost)',
'avg(best_offer_available)', 'max(best_offer_available)', 'min(best_offer_available)', 'stddev(best_offer_available)',
'avg(returns_allowed)', 'max(returns_allowed)', 'min(returns_allowed)', 'stddev(returns_allowed)',
'count(item_condition)']

In [736]:
# to be able to view all the columns
pd.set_option('display.max_columns', 100)

In [741]:
round(summary_df.sort_values(by = ['sponsored', 'item_condition'], ascending = False),2)

,sponsored,item_condition,avg(seller_score),max(seller_score),min(seller_score),stddev(seller_score),avg(num_items_sold),max(num_items_sold),min(num_items_sold),stddev(num_items_sold),avg(item_price),max(item_price),min(item_price),stddev(item_price),avg(shipping_cost),max(shipping_cost),min(shipping_cost),stddev(shipping_cost),avg(best_offer_available),max(best_offer_available),min(best_offer_available),stddev(best_offer_available),avg(returns_allowed),max(returns_allowed),min(returns_allowed),stddev(returns_allowed),count(item_condition)
0,1,Used,17273.4242,258795,0,44379.23,26.8788,1690,0,206.42,26227.2121,55351,15000,7805.77,408.3714,4090,0,913.99,0.5303,1,0,0.50,0.5303,1,0,0.50,132
3,1,Seller refurbished,15842.0000,41451,0,15258.84,348.2000,1737,0,694.40,22813.6000,25300,19999,1943.15,0.0000,0,0,0.00,0.4000,1,0,0.49,0.8000,1,0,0.40,10
4,1,Open box,15819.6000,64480,5,24974.39,1.0000,5,0,2.00,26060.0000,33500,21000,4337.56,750.0000,1500,0,750.00,0.6000,1,0,0.49,0.4000,1,0,0.49,10
2,1,New,34961.4130,916791,0,155293.22,13.8478,313,0,48.61,32298.6087,57999,22100,8818.45,402.8824,4999,0,1181.47,0.2609,1,0,0.44,0.7174,1,0,0.45,92
1,1,Manufacturer refurbished,294.5000,533,56,238.50,1.5000,3,0,1.50,26949.5000,33900,19999,6950.50,0.0000,0,0,0.00,1.0000,1,1,0.00,0.5000,1,0,0.50,4
5,0,Used,51318.7711,224421,0,74776.60,0.1295,17,0,1.23,22654.9437,50000,8500,5084.40,407.4164,4660,0,840.60,0.4128,1,0,0.49,0.5009,1,0,0.50,1066
9,0,Seller refurbished,312811.7500,2265043,0,742315.99,48.6563,1316,0,228.47,23092.0000,36995,11200,5174.40,278.2083,2999,0,762.45,0.3125,1,0,0.46,0.5938,1,0,0.49,64
8,0,Open box,15175.7759,568351,0,77237.71,0.1724,5,0,0.77,26350.1207,52995,12500,7123.89,758.5882,5700,0,1483.86,0.5517,1,0,0.50,0.2069,1,0,0.41,116
6,0,New,13088.1786,638464,-1,47683.85,17.3681,1896,0,140.30,31965.2665,126999,14300,11013.31,327.7580,4854,0,884.55,0.3077,1,0,0.46,0.5110,1,0,0.50,728
7,0,Manufacturer refurbished,453019.2000,2265041,0,906010.90,1.0000,5,0,2.00,21359.8000,25000,19000,2148.94,0.0000,0,0,0.00,0.8000,1,0,0.40,0.4000,1,0,0.49,10


## g)
<b> Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))

- sponsored items tend to have a higher seller score for new items, and lower seller score for items in all other conditions.
- sponsored items also have more items sold on average, especially for seller refurbished items.
- on average, sponsored items have higher item prices for new, used, manufacturer refurbished items, and lower price for items in other conditions.
- on average, more sponsored items provide the option of best offers and allow returns.
- the best variable that could be predict if the item was sponsored or not could be best offer available and returns allowed. Also, seller score in conjunction with item condition would could be a predictor. High seller score for new items would indicate an item to be sponsored.
